In [41]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from pathlib import Path
import joblib

In [42]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
file_path = Path("combined_final.csv")
ev_df = pd.read_csv(file_path)


# Review the DataFrame
ev_df.head()

,City,State,Total EV Chargers,Median Income (16 and over),Total Population
0,Abilene,Texas,23,35115,128149
1,Akron,Ohio,80,36969,188498
2,Alameda,California,145,82773,76039
3,Albany,Georgia,554,26766,67922
4,Albany,New York,554,26766,100832


In [43]:
# Defining a threshold for the number of chargers to determine EV station presence
charger_threshold = 10

In [44]:
# Creating a new column 'EV_Station_Presence' based on the threshold
ev_df['EV_Station_Presence'] = ev_df['Total EV Chargers'].apply(lambda x: 1 if x >= charger_threshold else 0)

In [45]:
# Feature columns used for prediction (including City)
feature_columns = ['Total EV Chargers', 'Median Income (16 and over)', 'Total Population', 'City']

In [46]:
# Creating feature matrix X and target vector y
X = ev_df[feature_columns]
y = ev_df['EV_Station_Presence']

In [47]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
# Preprocessing numerical features (scaling)
num_columns = ['Total EV Chargers', 'Median Income (16 and over)', 'Total Population']
scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train[num_columns])
X_test_num = scaler.transform(X_test[num_columns])

In [49]:
# Preprocessing categorical feature (one-hot encoding)
cat_column = ['City']
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_train_cat = encoder.fit_transform(X_train[cat_column])
X_test_cat = encoder.transform(X_test[cat_column])

D:\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [50]:
# Concatenating numerical and categorical features
X_train_final = Concatenate()([X_train_num, X_train_cat])
X_test_final = Concatenate()([X_test_num, X_test_cat])

In [51]:
# Building the neural network model with modifications as discussed
model = Sequential()
model.add(Dense(units=32, activation='relu', input_dim=X_train_final.shape[1]))  # Example reduction in model complexity
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [52]:
# Training the model with early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit(X_train_final, y_train, epochs=100, batch_size=32, verbose=1, validation_data=(X_test_final, y_test), callbacks=[early_stop])

Epoch 1/100
16/16 [==============================] - 1s 9ms/step - loss: -55.6449 - accuracy: 0.0000e+00 - val_loss: -96.3162 - val_accuracy: 0.0000e+00
Epoch 2/100
16/16 [==============================] - 0s 3ms/step - loss: -90.2794 - accuracy: 0.0000e+00 - val_loss: -149.4448 - val_accuracy: 0.0000e+00
Epoch 3/100
16/16 [==============================] - 0s 3ms/step - loss: -146.0060 - accuracy: 0.0000e+00 - val_loss: -212.7989 - val_accuracy: 0.0000e+00
Epoch 4/100
16/16 [==============================] - 0s 3ms/step - loss: -188.1988 - accuracy: 0.0000e+00 - val_loss: -266.4677 - val_accuracy: 0.0000e+00
Epoch 5/100
16/16 [==============================] - 0s 3ms/step - loss: -217.4671 - accuracy: 0.0000e+00 - val_loss: -334.1426 - val_accuracy: 0.0000e+00
Epoch 6/100
16/16 [==============================] - 0s 3ms/step - loss: -316.7135 - accuracy: 0.0000e+00 - val_loss: -405.8161 - val_accuracy: 0.0000e+00
Epoch 7/100
16/16 [==============================] - 0s 3ms/step - loss: 

16/16 [==============================] - 0s 3ms/step - loss: -9429.1562 - accuracy: 0.0000e+00 - val_loss: -12297.2939 - val_accuracy: 0.0000e+00
Epoch 54/100
16/16 [==============================] - 0s 3ms/step - loss: -10000.0830 - accuracy: 0.0000e+00 - val_loss: -12736.7715 - val_accuracy: 0.0000e+00
Epoch 55/100
16/16 [==============================] - 0s 3ms/step - loss: -10525.5195 - accuracy: 0.0000e+00 - val_loss: -13181.5693 - val_accuracy: 0.0000e+00
Epoch 56/100
16/16 [==============================] - 0s 3ms/step - loss: -10258.3955 - accuracy: 0.0000e+00 - val_loss: -13650.6914 - val_accuracy: 0.0000e+00
Epoch 57/100
16/16 [==============================] - 0s 3ms/step - loss: -10810.7451 - accuracy: 0.0000e+00 - val_loss: -14076.3203 - val_accuracy: 0.0000e+00
Epoch 58/100
16/16 [==============================] - 0s 3ms/step - loss: -11791.3438 - accuracy: 0.0000e+00 - val_loss: -14578.6162 - val_accuracy: 0.0000e+00
Epoch 59/100
16/16 [==============================] - 

In [53]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_final, y_test)
print(f"Neural Network Accuracy: {accuracy}")

# Calculating validation metrics
y_pred = model.predict(X_test_final)
y_pred_classes = (y_pred > 0.5).astype(int)  # Convert predicted probabilities to classes

precision = precision_score(y_test, y_pred_classes)
recall = recall_score(y_test, y_pred_classes)
f1 = f1_score(y_test, y_pred_classes)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

4/4 [==============================] - 0s 2ms/step - loss: -40958.3672 - accuracy: 0.0000e+00
Neural Network Accuracy: 0.0
4/4 [==============================] - 0s 1ms/step


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [40]:
# Save the scaler
scaler_filename = 'scaler.pkl'
scaler_path = Path(scaler_filename)
joblib.dump(scaler, scaler_path)

# Save the encoder
encoder_filename = 'encoder.pkl'
encoder_path = Path(encoder_filename)
joblib.dump(encoder, encoder_path)

# Save the model
model_filename = 'ev_model.h5'
model_path = Path(model_filename)
model.save(model_path)

D:\Anaconda\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
